In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

2023-02-03:16:33:42,205 INFO     [workspace.py:291] Found the config file in: /config.json
2023-02-03:16:33:42,791 INFO     [clientbase.py:192] Created a worker pool for first use


Workspace name: udacitymlworkspace
Azure region: eastus
Subscription id: dce3a2e3-2f87-4a43-9fd7-d3a4bdeb704f
Resource group: udacityml


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "cpu-cluster"


#verify that cluster does not exist
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing cluster, use it.")
except ComputeTargetException:
    compute_name = os.environ.get('AML_COMPUTE_CLUSTER_NAME', cluster_name)
    compute_min_nodes = os.environ.get('AML_COMPUTE_CLUSTER_MIN_NODES', 0)
    compute_max_nodes = os.environ.get('AML_COMPUTE_CLUSTER_MIN_NODES', 4)

    vm_size = os.environ.get('AML_COMPUTE_CLUSTER_SKU', 'Standard_D2_V2')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size=vm_size, min_nodes= compute_min_nodes, max_nodes=compute_max_nodes)

    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)

compute_target.wait_for_completion(show_output=True)


Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [6]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
        "C": uniform(0.5, 1.0),
        "max_iter": choice([5,100])
    }
)

# Specify a Policy

policy = BanditPolicy(slack_factor=0.1, evaluation_interval=2, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
 
src = ScriptRunConfig(source_directory='.',
                            script='train.py',
                            compute_target=compute_target,
                            environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling = ps,
                                     policy=policy,
                                     primary_metric_name="Accuracy",
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=4,
                                     max_concurrent_runs=4)

In [7]:

# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config)
hyperdrive_run.get_status()
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_e36d8ac7-d414-4302-8558-6027fc96b512
Web View: https://ml.azure.com/runs/HD_e36d8ac7-d414-4302-8558-6027fc96b512?wsid=/subscriptions/dce3a2e3-2f87-4a43-9fd7-d3a4bdeb704f/resourcegroups/udacityml/workspaces/udacitymlworkspace&tid=8c9a0d04-e250-4aeb-85f4-1b71f173e263

Streaming azureml-logs/hyperdrive.txt

[2023-02-03T03:23:18.105733][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-02-03T03:23:18.8729311Z][SCHEDULER][INFO]Scheduling job, id='HD_e36d8ac7-d414-4302-8558-6027fc96b512_0' 
[2023-02-03T03:23:18.9489322Z][SCHEDULER][INFO]Scheduling job, id='HD_e36d8ac7-d414-4302-8558-6027fc96b512_1' 
[2023-02-03T03:23:19.2542010Z][SCHEDULER][INFO]Scheduling job, id='HD_e36d8ac7-d414-4302-8558-6027fc96b512_3' 
[2023-02-03T03:23:19.0975714Z][SCHEDULER][INFO]Scheduling job, id='HD_e36d8ac7-d414-4302-8558-6027fc96b512_2' 
[2023-02-03T03:23:19.4929474Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_e36d8ac7-d414-4302-8558-6027fc96b512_0' 
[2023-02-03T03

{'runId': 'HD_e36d8ac7-d414-4302-8558-6027fc96b512',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2023-02-03T03:23:17.443495Z',
 'endTimeUtc': '2023-02-03T03:33:26.024467Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '5ef019fa-607d-446c-acb3-1091f6a4beda',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1022-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.47.0',
  'space_size': 'infinite_space_size',
  'score': '0.9108883575501195',
  'best_child_run_id': 'HD_e36d8ac7-d414-4302-8558-6027fc96b512_2',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_e36d8ac7-d414-4302-8558-6027fc96b512_2'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValu

KeyError: 'log_files'

In [8]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_metrics())
print(best_run.get_file_names())

model = best_run.register_model(model_name='model_hyperdrive', model_path='outputs/model.joblib')

best_run.download_file(best_run.get_file_names()[3], output_file_path='./outputs/')

joblib.load('./outputs/model.joblib')


{'Regularization Strength:': 0.9988227783811898, 'Max iterations:': 100, 'Accuracy': 0.9108883575501195}
['logs/azureml/dataprep/0/backgroundProcess.log', 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/0/rslex.log.2023-02-03-03', 'outputs/.amlignore', 'outputs/model.joblib', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/metrics_capability/metrics-capability.log', 'system_logs/snapshot_capability/snapshot-capability.log', 'user_logs/std_log.txt']


Trying to unpickle estimator LogisticRegression from version 0.24.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.


LogisticRegression(C=0.6140363903853683, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [4]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [5]:
import sys
sys.path.append('train')
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(data)

In [6]:
import pandas as pd
from azureml.core import Dataset

#joining x and y datframes into one dataframe
data = x.join(y)

#create a folder to save data file
if not os.path.isdir('data'):
    os.mkdir('data')
    
# Save the train data to a csv to be uploaded to the datastore
pd.DataFrame(data).to_csv("data/train_data.csv", index=False)

# setting the default datastore
ds = ws.get_default_datastore()
ds.upload(src_dir='./data', target_path='bankmarketing', overwrite=True, show_progress=True)
 
# Upload the training data as a tabular dataset for access during training on remote compute
train_data = Dataset.Tabular.from_delimited_files(path=ds.path('bankmarketing/train_data.csv'))
label = "y"

2023-02-03:16:35:16,975 INFO     [datastore_client.py:991] <azureml.core.authentication.InteractiveLoginAuthentication object at 0x7f62531a5160>
2023-02-03:16:35:17,722 WARNING  [_dataset_deprecation.py:89] "Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.
2023-02-03:16:35:17,723 INFO     [azure_storage_datastore.py:923] Called AzureBlobDatastore.upload
2023-02-03:16:35:17,742 INFO     [azure_storage_datastore.py:372] Uploading an estimated of 3 files


Uploading an estimated of 3 files
Uploading ./data/.amlignore
Uploaded ./data/.amlignore, 1 files out of an estimated total of 3
Uploading ./data/.amlignore.amltmp
Uploaded ./data/.amlignore.amltmp, 2 files out of an estimated total of 3
Uploading ./data/train_data.csv
Uploaded ./data/train_data.csv, 3 files out of an estimated total of 3
Uploaded 3 files


In [11]:
from azureml.train.automl import AutoMLConfig

# Setting parameters for AutoMLConfig

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='AUC_weighted',
    training_data=train_data,
    label_column_name=label,
    n_cross_validations=5,
    iteration_timeout_minutes=5,
    enable_onnx_compatible_models=True)


In [12]:
# Submitting automl run

from azureml.core.experiment import Experiment

experiment = Experiment(ws, "automl_test_experiment")
automl_run = experiment.submit(config=automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one cl

2023/02/03 16:45:44 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2023-02-03:16:54:43,859 INFO     [explanation_client.py:334] Using default datastore for uploads


In [13]:
# Retrieve and save your best automl model.
from azureml.automl.runtime.onnx_convert import OnnxConverter

best_automl_model, onnx_model= automl_run.get_output(return_onnx_model=True)

OnnxConverter.save_onnx_model(onnx_model, file_path="./automl_model.onnx")

run.complete()